### 기본 재무 데이터 비율 환산 

In [ ]:
import pandas as pd 
data = pd.read_csv('초기 데이터.csv')

In [ ]:
data["시총"] = data["시총"]/1000

# 안정성
data['유동비율'] = data['유동자산'] / data['유동부채'] * 100

data['당좌비율'] = data['당좌자산'] / data['유동부채'] * 100

data['부채비율'] = data['부채'] / data['자본'] * 100

data['차입금의존도'] = (data['장기차입금']+data['단기차입금']+data['사채']) / data['자산'] * 100

data['이자보상배율'] = data['영업이익']/ data['이자비용']


# 수익성
data['매출총이익률'] = data['매출총이익'] / data['매출액'] * 100

data['영업이익률'] = data['영업이익'] / data['매출액'] * 100

data['당기순이익률'] = data['당기순이익'] / data['매출액'] * 100

data['ROA'] = data['당기순이익'] / data['자산'] * 100

average_equity = data['자본'].rolling(2).mean() # 평균 자기 자본
data['ROE'] = data['당기순이익'] / average_equity * 100



# 활동성
inventory_equity = data['재고자산'].rolling(2).mean() # 평균 자기 자본
data['재고자산회전율'] = data['매출원가'] / inventory_equity

trade_receivable_equity = data['매출채권'].rolling(2).mean() # 평균 자기 자본
data['매출채권회전율'] = data['매출액'] / trade_receivable_equity

total_assets_equity = data['자산'].rolling(2).mean() # 평균 자기 자본
data['총자산회전율'] = data['매출액'] / total_assets_equity

trade_payables_equity = data['매입채무'].rolling(2).mean() # 평균 자기 자본
data['매입채무회전율'] = data['매출원가'] / trade_payables_equity

data["현금순환주기"] = ((360/data["재고자산회전율"])+(360/data["매출채권회전율"]))-(360/data["총자산회전율"])


# 성장성
data['매출액증가율'] = (data['매출액'].diff() / data['매출액'].shift(1)) * 100 

data['영업이익증가율'] = (data['영업이익'].diff() / data['영업이익'].shift(1)) * 100

data['유형자산증가율'] = (data['유형자산'].diff() / data['유형자산'].shift(1)) * 100

data['총자산증가율'] = (data['자산'].diff() / data['자산'].shift(1) * 100)

### 기초 데이터 기반으로 계산하는 기업별 비재무데이터 

In [ ]:
data = pd.read_excel('./진짜진짜최종.xlsx')
data.fillna(0, inplace=True)
#전처리
data['복리후생비/영업이익'] = data['복리후생비'] / data['영업이익']
data['기부금/영업이익'] = data['기부금'] / data['영업이익']
from sklearn.preprocessing import MinMaxScaler
import numpy as np
data['기업존속연수'] = data['회계년도'].astype(int) - (data['설립일'].dt.year).astype(int)
data['상장년수'] = data['회계년도'].astype(int) - (data['상장일'].dt.year).astype(int)
'''
scaler = MinMaxScaler()
aa = data['기업존속연수'].values.reshape(-1,1)
bb = data['상장년수'].values.reshape(-1,1)
new_aa = scaler.fit_transform(aa)
new_bb = scaler.fit_transform(bb)
data['기업존속연수'] = pd.DataFrame(new_aa)+1
data['상장년수'] = pd.DataFrame(new_bb)+1
'''
data['new_시총'] = data['시총'] / data['자본']
data['new_평균근속연수'] = (data['평균근속연수']/data['평균근속연수'].max()) / ((data['기업존속연수']/data['기업존속연수'].max()) + (data['상장년수']/data['상장년수'].max()))
data['new_기업존속연수'] = data['기업존속연수']/data['기업존속연수'].max()
data['new_상장년수'] = data['상장년수']/data['상장년수'].max()
data['자산/기말인원'] = data['자산'] / data['기말인원']
data['교육훈련비/영업이익'] = data['교육훈련비'] / data['영업이익']
data['경상개발비/영업이익'] = data['경상개발비'] / data['영업이익']


```python
data = pd.read_excel('./진짜진짜최종.xlsx')
data.fillna(0, inplace=True)
#전처리
data['복리후생비/영업이익'] = data['복리후생비'] / data['영업이익']
data['기부금/영업이익'] = data['기부금'] / data['영업이익']
from sklearn.preprocessing import MinMaxScaler
import numpy as np
data['기업존속연수'] = data['회계년도'].astype(int) - (data['설립일'].dt.year).astype(int)
data['상장년수'] = data['회계년도'].astype(int) - (data['상장일'].dt.year).astype(int)
'''
scaler = MinMaxScaler()
aa = data['기업존속연수'].values.reshape(-1,1)
bb = data['상장년수'].values.reshape(-1,1)
new_aa = scaler.fit_transform(aa)
new_bb = scaler.fit_transform(bb)
data['기업존속연수'] = pd.DataFrame(new_aa)+1
data['상장년수'] = pd.DataFrame(new_bb)+1
'''
data['new_시총'] = data['시총'] / data['자본']
data['new_평균근속연수'] = (data['평균근속연수']/data['평균근속연수'].max()) / ((data['기업존속연수']/data['기업존속연수'].max()) + (data['상장년수']/data['상장년수'].max()))
data['new_기업존속연수'] = data['기업존속연수']/data['기업존속연수'].max()
data['new_상장년수'] = data['상장년수']/data['상장년수'].max()
data['자산/기말인원'] = data['자산'] / data['기말인원']
data['교육훈련비/영업이익'] = data['교육훈련비'] / data['영업이익']
data['경상개발비/영업이익'] = data['경상개발비'] / data['영업이익']
data.to_excel('./최종_27.xlsx')
```

### 알트만 Z _score , 알트만 K1_score, 즘쥬스키 score 

In [ ]:
data.sort_values(by = ['거래소코드', '회계년도'], inplace = True)

#Z_score 계산식 
data["x1"] = (data["매출채권"]+data["재고자산"]-data["매입채무"])/data["자산"]
data["x2"]=data["이익잉여금"]/data["자산"]
data["x3"] = (data["당기순이익"]+data["법인세비용"]-data["이자수익"]+data["이자비용"]-data["지분법이익"]+data["지분법손실"])/data["자산"]
data["x4"] = data["시총"]/data["부채"]
data["x5"] = data["매출액"]/data["자산"]
data["Z-Score"] = (1.2*data["x1"]) + (1.4*data["x2"])+(3.3*data["x3"])+(6.6*data["x4"])+(1*data["x5"])
data.drop(["x1", "x2", "x3","x4","x5"], axis=1, inplace=True)

z_score = [] 
for i in data['Z-Score']:
    if i<1.81:
        x = '부실'
        z_score.append(x)
    elif (i>= 1.81)&(i<=2.99):
        x= '판정보류'
        z_score.append(x)
    elif i>2.99:
        x='건전'
        z_score.append(x)
    else:
        x='모름'
        z_score.append(x)
        
data['Z-Score_ad'] = z_score

# k1-score 계산

data["x1"] = data["자산"]/1000
data["x2"]=(data["매출액"]/1000)/(data["자산"]/1000)
data["x3"] = (data["이익잉여금"]/1000)/(data["자산"]/1000)
data["x4"] = (data["자본"]/1000)/(data["부채"]/1000)

data["x1"] = np.log1p(data["x1"])
data["x2"] = np.log1p(data["x2"])

data["K1_Score"] = -17.862+(1.472*data["x1"])+(3.041*data["x2"])+(14.839*data["x3"])+(1.516*data["x4"])

k_score = [] 
for i in data['K1_Score']:
    if i<-2.00:
        x = '부실'
        k_score.append(x)
    elif (i>= -2.00)&(i<=0.75):
        x= '판정보류'
        k_score.append(x)
    elif i>0.75:
        x='건전'
        k_score.append(x)
    else:
        x='모름'
        k_score.append(x)

data['k1-Score_ad'] = k_score
data.drop(["x1", "x2", "x3","x4"], axis=1, inplace=True)



data1s=data["Z-Score_ad"].to_list()
data2s=data["k1-Score_ad"].to_list()


and_score = [] 
for i in range(0,len(data1s)):
    if (data1s[i]=="부실") & (data2s[i]=="부실"):
        x = '부실'
        and_score.append(x)
    elif (data1s[i]=="부실") & (data2s[i]=="건전"):
        x= '건전'
        and_score.append(x)
    elif (data1s[i]=="건전") & (data2s[i]=="부실"):
        x= '건전'
        and_score.append(x)
    elif (data1s[i]=="부실") & (data2s[i]=="판정보류"):
        x='판정보류'
        and_score.append(x)
    elif (data1s[i]=="건전") & (data2s[i]=="건전"):
        x='건전'
        and_score.append(x)
    elif (data1s[i]=="건전") & (data2s[i]=="판정보류"):
        x='판정보류'
        and_score.append(x)
    elif (data1s[i]=="판정보류") & (data2s[i]=="판정보류"):
        x='판정보류'
        and_score.append(x)
    elif (data1s[i]=="판정보류") & (data2s[i]=="건전"):
        x='판정보류'
        and_score.append(x)
    elif (data1s[i]=="판정보류") & (data2s[i]=="부실"):
        x='판정보류'
        and_score.append(x)
    else:
        x='모름'
        and_score.append(x)
set(and_score)

or_score = [] 
for i in range(len(data1s)):
      if (data1s[i]=="부실") & (data2s[i]=="부실"):
            y = '부실'
            or_score.append(y)
      elif (data1s[i]=="건전") & (data2s[i]=="건전"):
            y='건전'
            or_score.append(y)   
      elif (data1s[i]=="부실") & (data2s[i]=="건전"):
            y= '부실'
            or_score.append(y)
      elif (data1s[i]=="건전") & (data2s[i]=="부실"):
            y= '부실'
            or_score.append(y)
      elif (data1s[i]=="부실") & (data2s[i]=="판정보류"):
            y='판정보류'
            or_score.append(y)

      elif (data1s[i]=="건전") & (data2s[i]=="판정보류"):
            y='판정보류'
            or_score.append(y)
      elif (data1s[i]=="판정보류") & (data2s[i]=="판정보류"):
            y='판정보류'
            or_score.append(y)
      elif (data1s[i]=="판정보류") & (data2s[i]=="건전"):
            y='판정보류'
            or_score.append(y)
      elif (data1s[i]=="판정보류") & (data2s[i]=="부실"):
            y='판정보류'
            or_score.append(y)
      else:
            y='모름'
            or_score.append(y)
            
data['종합 and']=np.nan
data['종합 or'] =np.nan
data['종합 and']=and_score
data['종합 or'] =or_score